In [7]:
!pip install dash
!pip install wget


In [10]:
import wget
wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py")

'spacex_dash_app.py'

In [11]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                             options=[ {'label':'All','value':'All'},
                                                       {'label':'CCAFS LC-40','value':'CCAFS LC-40'},
                                                       {'label':'VAFB SLC-4E','value':'VAFB SLC-4E'},
                                                       {'label':'KSC LC-39A','value':'KSC LC-39A'},
                                                       {'label':'CCAFS SLC-9','value':'CCAFS SLC-9'}],
                                            value='ALL',
                                            placeholder="Select a Launce Site Here",
                                            searchable=True
                                             ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',min=0,max=10000,step=1000,
                                                marks = {0:'0',1000:'100','2000':2000,'5000':5000,'10000':10000},
                                                value=[min_payload,max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart',component_property='figure'),
              Input(component_id='site-dropdown',component_property='value')
              )
def get_pie_chart(entered_site):
    df = spacex_df
    if entered_site =='All':
        fig = px.pie(df,values='class',names="Launch Site",title='All Sites')
        return fig
    else:
        fig = px.pie(df[df["Launch Site"]==entered_site],values='Unnamed: 0',names="class",title=entered_site)
        return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart',component_property='figure'),
              [Input(component_id='site-dropdown',component_property='value'),
              Input(component_id='payload-slider',component_property='value')
              ]
)

def get_scatter_chart(entered_site,payload_range):
  
    if entered_site =='All':
        df = spacex_df[(spacex_df['Payload Mass (kg)'] >= int(payload_range[0])) &
                        (spacex_df['Payload Mass (kg)'] <= int(payload_range[1]))]
        fig = px.scatter(df,x="Payload Mass (kg)",y='class',color="Booster Version Category")
        return fig
    else:
        df = spacex_df[(spacex_df["Launch Site"]==entered_site) &
                       (spacex_df['Payload Mass (kg)'] >= int(payload_range[0])) &
                        (spacex_df['Payload Mass (kg)'] <= int(payload_range[1]))]
        fig = px.scatter(df,x="Payload Mass (kg)",y='class',color="Booster Version Category")
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
